In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import Sequence
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from keras import layers
from keras import models
from keras.models import Model

In [34]:

# Define a custom data generator that inherits from the Sequence class
class CustomDataGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size, image_size):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_samples = len(image_paths)

    def __len__(self):
        # Calculate the number of batches needed to cover all samples
        return int(np.ceil(self.num_samples / self.batch_size))

    def __getitem__(self, index):
        #Get the indices for the current batch
        start = index * self.batch_size
        end = (index + 1) * self.batch_size
        batch_image_paths = self.image_paths[start:end]
        batch_labels = self.labels[start:end]
        batch_images = [self.load_image(image_path) for image_path in batch_image_paths]

        return np.array(batch_images), np.array(batch_labels)

    def load_image(self, image_path):
        
        image = plt.imread(image_path)
        #image = self.random_crop(image, self.image_size)
        image = tf.image.resize(image, self.image_size)
        #The images are converted from RGB to BGR, then each color channel is zero-centered with respect to the ImageNet dataset, without scaling.
        image = tf.keras.applications.efficientnet.preprocess_input(image)    

        #image = tf.keras.applications.vgg16.preprocess_input(image)
        #image = preprocess_input(image)
        return image

    def random_crop(self, image, crop_size):
        # Randomly crop the image
        height, width, _ = image.shape
        dy, dx = crop_size
        x = np.random.randint(0, width - dx + 1)
        y = np.random.randint(0, height - dy + 1)
        cropped_image = image[y:(y+dy), x:(x+dx), :]
        
        return cropped_image

In [35]:
batch_size = 4
image_size = (380, 380)  # Adjust the image size based on your model's input requirements
num_classes = 7  # Number of classes in your classification task

In [36]:
# Initialize the EfficientNetB3 model
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=image_size + (3,))

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [40]:
# Define the root directory where your data is stored
root_directory = "crop_dataset_2"

# Get the list of subdirectories (categories)
subdirectories = [subdir for subdir in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory, subdir))]

# Create lists to store image paths and corresponding labels
image_paths = []
labels = []

# Iterate through each subdirectory (category)
for label, subdirectory in enumerate(subdirectories):
    # Construct the full path to the subdirectory
    subdirectory_path = os.path.join(root_directory, subdirectory)
    
    # Get a list of image files in the subdirectory
    image_files = [os.path.join(subdirectory_path, filename) for filename in os.listdir(subdirectory_path) if filename.endswith('.png')]  # Adjust the file extension as needed
    
    # Append image paths and labels
    image_paths.extend(image_files)
    labels.extend([label] * len(image_files))


# Now you have the image_paths and labels
print("Total images:", len(image_paths))
print("Total labels:", len(labels))

Total images: 1864
Total labels: 1864


In [41]:
# Create custom data generator
data_generator = CustomDataGenerator(image_paths, labels, batch_size, image_size)

In [43]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model using the custom data generator
model.fit(data_generator, epochs=10)  # Adjust the number of epochs as needed

<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
Epoch 1/10


KeyboardInterrupt: 